# Bouncing Ball with Manim

Run the cells below to render a bouncing ball animation using Manim inside Jupyter.


In [1]:
%load_ext manim.utils.ipython_magic

/Users/lucasstarkey/Projects/ForFun/manim-sandbox/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


The manim.utils.ipython_magic module is not an IPython extension.


In [2]:
%%manim -qm -v WARNING BouncingBallScene
from manim import *

class BouncingBallScene(Scene):
    def construct(self):
        ground = Line(LEFT * 5, RIGHT * 5).shift(DOWN * 3)
        ball = Circle(radius=0.3, color=BLUE).set_fill(BLUE, opacity=0.9).move_to(UP * 3)
        shadow = Circle(radius=0.22, color=GRAY_E, fill_opacity=0.4).move_to(DOWN * 2.7)

        self.play(Create(ground), FadeIn(shadow), FadeIn(ball))
        self.wait(0.2)

        peaks = [UP * 3, UP * 1.8, UP * 1.1, UP * 0.6]
        floor = DOWN * 2.7

        for peak in peaks:
            self.play(ball.animate.move_to(floor), shadow.animate.scale(1.4),
                      run_time=0.5, rate_func=rate_functions.ease_in)
            self.play(ball.animate.move_to(peak), shadow.animate.scale(0.7),
                      run_time=0.6, rate_func=rate_functions.ease_out)
            shadow.scale(1.0 / 0.7)  # reset scaling for the next bounce

        self.play(FadeOut(ball), FadeOut(shadow), FadeOut(ground))
        self.wait(0.2)


Manim Community v0.19.0

AttributeError: module 'manim.utils.rate_functions' has no attribute 'ease_in'